# Getting Started with the Enterprise Knowledge Assistant

This notebook demonstrates how to use the Enterprise Knowledge Assistant, a production-ready LangGraph application that combines all the concepts covered in the LangGraph mastery curriculum.

## Setup

First, let's set up our environment and import the necessary modules.

In [ ]:
import os
import sys
from dotenv import load_dotenv

# Add the parent directory to the path so we can import from the root
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))

# Load environment variables from .env file
load_dotenv()

# Verify that the GOOGLE_API_KEY is set
if not os.getenv("GOOGLE_API_KEY"):
    print("Warning: GOOGLE_API_KEY is not set. Please set it in the .env file.")

## Initialize the Enterprise Knowledge Assistant

Now, let's initialize the Enterprise Knowledge Assistant by creating the main workflow graph.

In [ ]:
from src.main import initialize_app

# Initialize the application
graph = initialize_app()
print("Enterprise Knowledge Assistant initialized successfully!")

## Process a Query

Let's process a query through the Enterprise Knowledge Assistant.

In [ ]:
from src.main import process_query

# Process a query
query = "What is the company policy on data security?"
result = process_query(graph, query)

# Display the response
if "error" in result:
    print(f"Error: {result['error']}")
elif "results" in result and "final_response" in result["results"]:
    print(f"Response: {result['results']['final_response']}")
else:
    print("No response generated.")

## Examine the Workflow State

Let's examine the workflow state to understand how the query was processed.

In [ ]:
import json

# Print the results
print("Results:")
for key, value in result.get("results", {}).items():
    if key != "final_response":  # We already displayed this
        print(f"\n{key}:")
        print(json.dumps(value, indent=2))

## Provide Feedback

Let's provide feedback on the response, which will be processed by the feedback collection node.

In [ ]:
# Process a query with feedback
query = "What are the steps to request access to the ERP system?"
context = {
    "feedback": {
        "rating": 4,
        "comments": "Good response, but could provide more details on the approval process."
    }
}
result = process_query(graph, query, context)

# Display the response
if "results" in result and "final_response" in result["results"]:
    print(f"Response: {result['results']['final_response']}")

# Check if feedback was processed
if "results" in result and "feedback_collection" in result["results"]:
    print("\nFeedback processed:")
    print(json.dumps(result["results"]["feedback_collection"], indent=2))

## Error Handling

Let's see how the system handles errors by simulating an error condition.

In [ ]:
from src.models.state import AssistantState, ErrorRecord, ErrorSeverity

# Create a state with an error
state = AssistantState(
    query="What is the company policy on data security?",
    context={},
    messages=[],
    current_node="query_understanding",
    results={},
    errors=[
        ErrorRecord(
            message="Simulated error for demonstration",
            node="query_understanding",
            severity=ErrorSeverity.ERROR,
            details={"query": "What is the company policy on data security?"}
        )
    ],
    metadata={"recovery_attempts": 0},
)

# Process the state through the graph
result = graph.invoke(state.model_dump())

# Display the error handling results
if "results" in result and "error_handling" in result["results"]:
    print("Error handling results:")
    print(json.dumps(result["results"]["error_handling"], indent=2))
    
    # Display the user message
    print("\nUser message:")
    print(result["results"]["error_handling"]["user_message"])

## Memory Management

Let's examine the conversation history stored in memory.

In [ ]:
from src.graphs.nodes.memory_management import get_conversation_history

# Get the conversation history
history = get_conversation_history(limit=5)

# Display the conversation history
print(f"Found {len(history)} conversation entries in memory:")
for i, entry in enumerate(history, 1):
    print(f"\nEntry {i}:")
    print(f"Query: {entry.get('query')}")
    print(f"Response: {entry.get('response')[:100]}..." if entry.get('response') else "No response")
    if entry.get('feedback'):
        print(f"Feedback: Rating {entry['feedback'].get('rating')}/5")

## Conclusion

This notebook demonstrated how to use the Enterprise Knowledge Assistant, a production-ready LangGraph application. We covered:

1. Initializing the assistant
2. Processing queries
3. Examining the workflow state
4. Providing feedback
5. Handling errors
6. Managing memory

These concepts build on the fundamentals covered in the LangGraph mastery curriculum, showing how they can be applied in a real-world enterprise setting.